In [1]:
from bs4 import BeautifulSoup

import requests

#first loop open the big page with url
#second loop find all links to a place 

url = "www.yelp.co.uk/search?find_loc=praha&cflt=restaurants"

r  = requests.get("http://" +url)

data = r.text

soup = BeautifulSoup(data,"lxml")

place_links = [] #store all sublinks of places

#find all links to places
for link in soup.find_all('a','biz-name'):
    place_links.append(link['href'])


place_pages = []


for place_link in place_links:
    tmpUrl = 'www.yelp.co.uk' + place_link    
    r  = requests.get("http://" +tmpUrl) #getting subpage
    data = r.text
    place_soup = BeautifulSoup(data,"lxml")
    place_pages.append(place_soup)
    
    

In [10]:
dataStorage = []

{'name':'',
 'address': '', 
 'priceRange': '', 
 'gps': [1,2], 
 'priceGroup': '', 
 'stars': '', 
 'numOfReviews': 0, 
 'web': '', 
 'numOfPhotos': '',
 'tags': [],
 'noiseLevel': '',
 'wifi': '',
 'openHours': {
        'mo' : '',
        'tu' : '',
        'we' : '',
        'th' : '',
        'fr' : '',
        'sa' : '',
        'su' : '',
    }
}

i = 1

name = place_pages[i].find('h1','biz-page-title').children.next().encode('utf-8') #need to encode utf-8 for saving into file

#address
address = place_pages[i].find('address').findChildren('span')
street = address[0].text
postCode = address[1].text
city = address[2].text

#price
priceRange = place_pages[i].find('dd','price-description')
priceText = priceRange.text.strip() #clean whitespaces
currency = priceText[:3]
priceText_split = priceText[3:].split('-') 
priceFrom = priceText_split[0]
priceTo = priceText_split[1]

priceGroup = place_pages[i].find('span','price-range')

#location
coordTmp = place_pages[i].find('a','biz-map-directions').findChildren()[0].attrs['src']
coordStart = coordTmp.find('center')
coordTmpShort = coordTmp[coordStart+7:coordStart+28]
gps = coordTmpShort.split('%2C')
gps = map(float,gps) #convert to int

[50.090748, 14.426998]


In [95]:
#saving into file
with open("Output.txt", "w") as text_file:
    text_file.write("Purchase Amount: {}".format(street))